In [4]:
include("StackyFan.jl")

minimalByLex (generic function with 1 method)

In [10]:
#inputs are a stacky fan and a vector of booleans representing the distinguished structure.
function BerghA(F::StackyFan,D::Array{Int64,1})
    X=deepcopy(F)
    rayMatrix=Array(X.fan.RAYS)
    coneList=convertIncidenceMatrix(X.fan.MAXIMAL_CONES)
    dim=size(rayMatrix,2)
    l=size(rayMatrix,1)
    #check if the vector D has length equal to the number of rays in F
    if l != size(D,1)
        error("length of vector representing distinguished structure does not agree with number of rays in stacky fan.")
    end
    
    #A0
    i=0
    while(true)
        #A1
        #Note: cones in S are 1-indexed.
        S=filter(cone->distinguishedAndMultiplicity(cone,rayMatrix,D),coneList)
        if S==[]
            break
        end
        
        #A2 - find extremal cones
        Smax=extremalCones(S,rayMatrix,D)
        
        #A2 - find interior points in Smax
        intPoints=[]
        for cone in Smax
            C=coneConvert(cone,rayMatrix)
            coneIntPoints=interiorPoints(C)
            for point in coneIntPoints
               push!(intPoints,(point,cone)) 
            end
        end
        
        #A2 - find stacky points (in terms of coefficients) derived from interior points
        P=[]
        for (point,cone) in intPoints
            stackyPoint=findStackyPoint(point,cone,rayMatrix,X.scalars)
            push!(P,stackyPoint)
        end
        
        #A2 - find smallest element of P with respect to lex ordering.
        psi=minimalByLex(P)
        
        
    end
    return X
end

BerghA (generic function with 1 method)

In [12]:
F=makeStackyFan([1 0;0 1],[[0,1]],[4,1])

StackyFan(Polymake.BigObjectAllocated(Ptr{Nothing} @0x000000001868eea0), [4, 1], Dict("0,1" => 1,"1,0" => 4))